In [ ]:
import os
import glob
import sqlite3
import pandas as pd

pasta = '../data'
tipo_arquivo = '*.json'

#Leitura dos aquivos em uma lista
caminhos_arquivos = glob.glob(os.path.join(pasta, tipo_arquivo))

In [ ]:
#Leitura e carga dos arquivos json no banco de dados SQLite
df = pd.DataFrame(columns=['customer', 'flow', 'session', 'timestamp', 'key', 'value'])

conn = sqlite3.connect('meta.db')

for arquivo in caminhos_arquivos:
    data = pd.read_json(arquivo)
    data['key'] = data.apply(lambda x: list(x['content'].keys())[0], axis=1)
    data['value'] = data.apply(lambda x: x['content'][x['key']], axis=1)
    data.drop(columns=['content'], inplace=True, errors='ignore')
    df = pd.concat([df, data], axis=0, ignore_index=False)

# Remove registros nulos e vazios
df['value'] = df['value'].apply(lambda x: None if x.strip() == '' else x)
df.dropna(subset=['value'], inplace=True)
df.head()

df.to_sql('chatbot', conn, if_exists='replace', index=False)
conn.close()

C:\Users\anselmo.oliveira\AppData\Local\Temp\ipykernel_48696\2574524264.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, data], axis=0, ignore_index=False)


In [4]:
#Teste do banco de dados
conn = sqlite3.connect('meta.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM chatbot")

data = cursor.fetchall()

select = pd.DataFrame(data, columns=['customer', 'flow', 'session', 'timestamp', 'key', 'value'])

conn.close()

select

,customer,flow,session,timestamp,key,value
0,C1000,F1000,S1000,2019-12-16 13:59:58+00:00,name,maria
1,C1000,F1000,S2000,2019-12-16 13:59:59+00:00,name,joao
2,C2000,F2000,S3000,2019-12-16 13:59:59+00:00,recomenda,Simmmmmmm
3,C1000,F1000,S2000,2019-12-16 13:59:59+00:00,cpf,733.600.420-26
4,C1000,F1000,S1000,2019-12-16 14:00:00+00:00,cpf,ZZZ
5,C1000,F1000,S2000,2019-12-16 14:00:00+00:00,delivery_confirmed,false
6,C1000,F1000,S1000,2019-12-16 14:00:01+00:00,cpf,305.584.960-40
7,C2000,F2000,S3000,2019-12-16 14:00:01+00:00,nota,9
8,C1000,F1000,S1000,2019-12-16 14:00:01+00:00,delivery_confirmed,true


In [5]:
# Exploração dos dados

conn = sqlite3.connect('meta.db')
cursor = conn.cursor()

data = cursor.execute("""WITH principal as (
               SELECT 
               ROW_NUMBER() OVER (PARTITION BY customer, flow, session, key ORDER BY timestamp DESC) AS row_number,
               customer, 
               flow, 
               session, 
               timestamp, 
               key, 
               value
               FROM chatbot
               WHERE 1=1
               ORDER BY timestamp ASC)
               
               SELECT customer, 
               flow, 
               session, 
               timestamp, 
               key, 
               value
               FROM principal
               WHERE row_number = 1
               ORDER BY customer, 
               flow, 
               session, 
               timestamp
               """)

data = cursor.fetchall()

df = pd.DataFrame(data, columns=['customer', 'flow', 'session', 'timestamp', 'key', 'value'])

conn.close()

In [61]:
def consulta_chatbot(customer, flow, session):
    #Teste do banco de dados
    conn = sqlite3.connect('meta.db')
    cursor = conn.cursor()
    cursor.execute(f"""WITH principal as (
               SELECT ROW_NUMBER() OVER (PARTITION BY customer, flow, session, key ORDER BY timestamp DESC) AS row_number,
               customer, 
               flow, 
               session, 
               timestamp, 
               key, 
               value
               FROM chatbot
               WHERE 1=1
               ORDER BY timestamp ASC),
                   
            periodo as (
            SELECT customer, 
                flow,  
                session, 
                min(timestamp) as first_answer_dt, 
                max(timestamp) as last_answer_dt
                FROM chatbot 
                where 1=1
                and flow = '{flow}'
                group by customer, flow, session)

            SELECT p.customer, 
                p.flow,  
                p.session, 
                pe.first_answer_dt as first_answer_dt, 
                pe.last_answer_dt last_answer_dt,  
                p.key,  
                p.value  
                FROM principal p
                left join periodo pe on p.customer = pe.customer and p.flow = pe.flow and p.session = pe.session
                where 1=1
                and p.row_number = 1
                and p.customer = '{customer}'
                and p.flow = '{flow}'
                and p.session = '{session}'
                group by p.customer, p.flow, p.session, p.key
                order by p.timestamp asc""")

    data = cursor.fetchall()
    select = pd.DataFrame(data, columns=['customer', 'flow', 'session', 'first_answer_dt', 'last_answer_dt', 'key', 'value'])
    conn.close()
    return select

In [99]:
data = consulta_chatbot(customer='C1000', flow='F1000', session='S1000')
result_1 = select[['customer', 'flow', 'session', 'first_answer_dt', 'last_answer_dt']].iloc[:1, :]
data = data[['key', 'value']].set_index('key', drop=True)
data = data.T.to_dict('records')
result_2 = pd.DataFrame(data)
result = pd.concat([result_1, result_2], axis=1)
result

,customer,flow,session,first_answer_dt,last_answer_dt,name,cpf,delivery_confirmed
0,C1000,F1000,S1000,2019-12-16 13:59:58+00:00,2019-12-16 14:00:01+00:00,maria,305.584.960-40,true
